In [6]:
from azureml.core import Workspace, Run, Dataset
from azureml.pipeline.wrapper import Pipeline, dsl, Module
from datetime import datetime

#ws = Workspace.get(name='kubeflow_ws_1', subscription_id='74eccef0-4b8d-4f83-b5f9-fa100d155b22', resource_group='kubeflow-demo')
ws = Workspace.get(name='itp-pilot', subscription_id='4aaa645c-5ae2-4ae9-a17a-84b9023bc56a', resource_group='itp-pilot-ResGrp')

modulefunc = Module.from_yaml(ws, yaml_file='./noop/1in2out.spec.yaml')
data = Dataset.get_by_name(ws, 'training_data')

In [9]:
@dsl.pipeline(
    name='A huge pipeline composed with nodes 1 in 2 outs',
    description='A sample',
    default_compute_target='k80-16-a'
)
def cell_division():
    layer = 11
    nodes = []
    nodes.append(modulefunc(input1=data))
    for i in range(0, layer-1):
        print('i=', i, ' nodes len=', len(nodes))
        current_layer_nodes = []
        for j in range(0, pow(2,i)):
            print(datetime.now().strftime("%d/%m/%Y %H:%M:%S"), '\tj=', j)
            n = nodes[-j-1]
            current_layer_nodes.append(modulefunc(input1=n.outputs.output1))
            current_layer_nodes.append(modulefunc(input1=n.outputs.output2))
        nodes = nodes + current_layer_nodes
    
    return {**nodes[-1].outputs}

pipeline = cell_division()

pipeline.submit(experiment_name='samples')

j= 61
29/05/2020 03:00:59 	j= 62
29/05/2020 03:00:59 	j= 63
29/05/2020 03:00:59 	j= 64
29/05/2020 03:00:59 	j= 65
29/05/2020 03:00:59 	j= 66
29/05/2020 03:00:59 	j= 67
29/05/2020 03:00:59 	j= 68
29/05/2020 03:00:59 	j= 69
29/05/2020 03:00:59 	j= 70
29/05/2020 03:00:59 	j= 71
29/05/2020 03:00:59 	j= 72
29/05/2020 03:00:59 	j= 73
29/05/2020 03:00:59 	j= 74
29/05/2020 03:00:59 	j= 75
29/05/2020 03:00:59 	j= 76
29/05/2020 03:00:59 	j= 77
29/05/2020 03:00:59 	j= 78
29/05/2020 03:00:59 	j= 79
29/05/2020 03:00:59 	j= 80
29/05/2020 03:00:59 	j= 81
29/05/2020 03:00:59 	j= 82
29/05/2020 03:00:59 	j= 83
29/05/2020 03:00:59 	j= 84
29/05/2020 03:00:59 	j= 85
29/05/2020 03:00:59 	j= 86
29/05/2020 03:00:59 	j= 87
29/05/2020 03:00:59 	j= 88
29/05/2020 03:00:59 	j= 89
29/05/2020 03:00:59 	j= 90
29/05/2020 03:00:59 	j= 91
29/05/2020 03:00:59 	j= 92
29/05/2020 03:00:59 	j= 93
29/05/2020 03:00:59 	j= 94
29/05/2020 03:00:59 	j= 95
29/05/2020 03:00:59 	j= 96
29/05/2020 03:00:59 	j= 97
29/05/2020 03:00:59 	j

Experiment,Id,Type,Status,Details Page,Docs Page
samples,bef38229-c05d-4862-89d4-a8af134dbd86,azureml.PipelineRun,NotStarted,Link to Azure Machine Learning studio,Link to Documentation
